# Makemore 4: Becoming a Backprop Ninja - Exercises

Notes on the exercises from the [makemore #4 video](https://www.youtube.com/watch?v=q8SA3rM6ckI).<br>
Feel free to try solving on your own first, using [this starter notebook](./005%20-%20Makemore_4_Exercises.ipynb).

1. Watch the [makemore #4 video](https://www.youtube.com/watch?v=q8SA3rM6ckI) on YouTube
2. Come back and solve these exercises to level up :)

In [1]:
# There's no functional change from last lecture in the first several cells

import torch
import random
import torch.nn.functional as F
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Read in all the names
words = open('../names.txt', 'r').read().splitlines()
print('Samples:', words[:8])
print('Size:', len(words))
print('Largest:', max(len(w) for w in words))

Samples: ['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']
Size: 32033
Largest: 15


In [3]:
# Build a vocabulary of characters map them to integers (these will be the index tokens)
chars = sorted(list(set(''.join(words))))  # set(): Throwing out letter duplicates
stoi = {s:i+1 for i,s in enumerate(chars)} # Make tupels of type (char, counter)
stoi['.'] = 0                              # Add this special symbol's entry explicitly
itos = {i:s for s,i in stoi.items()}       # Switch order of (char, counter) to (counter, char)
vocab_size = len(itos)

print(itos)
print(f'Total vocabulary size: {vocab_size}')

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
Total vocabulary size: 27


In [4]:
# Build the dataset
block_size = 3 # Context length: We look at this many characters to predict the next one

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

# Randomize the dataset (with reproducibility)
random.seed(42)
random.shuffle(words)

# These are the "markers" we will use to divide the dataset
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

# Dividing the dataset into train, dev and test splits
Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


Ok biolerplate done, now we get to the action:

In [5]:
# Utility function
# Compares your manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt==t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
n_embd = 10   # Dimensionality of the character embedding vectors
n_hidden = 64 # Number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647)
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(f'Total Parameter Count: {sum(p.nelement() for p in parameters)}') # total parameter count

for p in parameters:
  p.requires_grad = True

Total Parameter Count: 4137


**Note:** Many of these parameters are initialized in non-standard ways because sometimes initializating<br>
with e.g. all zeros could mask incorrect or slightly incorrect implementation of the backward pass.

In [7]:
batch_size = 32
# Shorter batch size variable, for convenience
n = batch_size
# Constructing a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X, Y

In [8]:
# Forward pass, "chunkated" into smaller steps that are possible to backprop through, one at a time

emb = C[Xb]                                      # Embed the characters into vectors
embcat = emb.view(emb.shape[0], -1)              # Concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1                        # Hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)        # Mean across over each hidden unit of the batch
bndiff = hprebn - bnmeani                        # Subtract mean from hidden layer pre-activated values
bndiff2 = bndiff**2                              # Square the differences between mean and hidden values (for variance)
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True)   # Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5                 # Inverted square root of the null-corrected variance
bnraw = bndiff * bnvar_inv                       # Normalize hidden layer pre-activated values
hpreact = bngain * bnraw + bnbias                # Scale and shift the normalized values
# Non-linearity
h = torch.tanh(hpreact)
# Linear layer 2
logits = h @ W2 + b2                             # Output layer
# Cross entropy loss (explicit here, but same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes               # Subtract max logit value for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1                  # If I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()

print(loss)

tensor(3.3361, grad_fn=<NegBackward0>)


## Exercise 1 - Backprop through the whole thing manually

**Objective:** Backpropagate through exactly all of the variables<br>
as they are defined in the forward pass above, one by one.

Below I condensed the code I implemented in the documenting notebook for comparing notes.<br>
If you'd like more detailed elaborations on why I implemented the following steps the way I did, refer to the documenting notebook [005 - Makemore #4.ipynb](./005%20-%20Makemore%20#4.ipynb).

In [9]:
## logprobs
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n
cmp('logprobs', dlogprobs, logprobs)

## probs
dprobs = (1 / probs) * dlogprobs
cmp('probs', dprobs, probs)

## counts_sum_inv
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

## counts - Step 1/2
dcounts = counts_sum_inv * dprobs

## counts_sum
dcounts_sum = -counts_sum**(-2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

## counts - Step 2/2
dcounts += torch.ones_like(counts) * dcounts_sum
cmp('counts', dcounts, counts)

## norm_logits
dnorm_logits = counts * dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

## logit_maxes
dlogits = dnorm_logits.clone()
dlogit_maxes = -1 * dnorm_logits.sum(1, keepdim=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)

## logits
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp('logits', dlogits, logits)

## layer 2 hidden values
dh = dlogits @ W2.T
cmp('h', dh, h)

## layer 2 weights
dW2 = h.T @ dlogits
cmp('W2', dW2, W2)

## layer 2 biases
db2 = dlogits.sum(0)
cmp('b2', db2, b2)

## hpreact
dhpreact = (1.0 - h ** 2) * dh
cmp('hpreact', dhpreact, hpreact)

## bngain
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
cmp('bngain', dbngain, bngain)

## bnbias
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('bnbias', dbnbias, bnbias)

## bnraw
dbnraw = bngain * dhpreact
cmp('bnraw', dbnraw, bnraw)

## bnvar_inv
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

## bndiff - Step 1/2
dbndiff = bnvar_inv * dbnraw

## bnvar
dbnvar = (-0.5 * (bnvar + 1e-5) ** (-1.5)) * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

## bndiff2
dbndiff2 = (1.0 / (n-1)) * torch.ones_like(bndiff2) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)

## bndiff - Step 2/2
dbndiff += 2 * bndiff * dbndiff2
cmp('bndiff', dbndiff, bndiff)

## bnmeani
dbnmeani = (-1.0) * dbndiff.sum(0, keepdim=True)
cmp('bnmeani', dbnmeani, bnmeani)

## hprebn
dhprebn = dbndiff.clone()
dhprebn += (1.0 / n) * torch.ones_like(hprebn) * dbnmeani
cmp('hprebn', dhprebn, hprebn)

## embcat
dembcat = dhprebn @ W1.T
cmp('embcat', dembcat, embcat)

## layer 1 weights
dW1 = embcat.T @ dhprebn
cmp('W1', dW1, W1)

## layer 1 biases
db1 = dhprebn.sum(0)
cmp('b1', db1, b1)

## emb
demb = dembcat.view(n, -1, emb.shape[2])
cmp('emb', demb, emb)

## C
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k, j]        # integer index of a character
        dC[ix] += demb[k, j] # add the gradient of the character embedding to the gradient of the character count
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
bngain          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnbias          | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09
bnraw  

The rounding errors are due to numerical instabilities.<br>
The documenting notebook [005 - Makemore #4.ipynb](./005%20-%20Makemore%20#4.ipynb) does not show these rounding errors, which is likely due to it having been built on Windows and with PyTorch 2.0.1.<br>
(The current notebook was built on Google Colab, i.e. Linux, with PyTorch 2.5.1)

I wouldn't like having to stick to any specific PyTorch or OS configuration for
the numbers to look nice and I will rather show you the real deal here, which means having some tiny inaccuracy.

## Exercise 2 - Backprop through `cross_entropy`, but all in one go

**Objective:** To complete this challenge, look at the mathematical expression of the loss,<br>
take the derivative, simplify the expression, and write it out.

In [10]:
# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.3360559940338135 diff: -2.384185791015625e-07


Ok, we are supposed to implement the gradient of the cross-entropy loss with respect to the logits.<br>
Cross-entropy loss is defined as: $L = -\sum_{i} y_i \log(\hat{y}_i)$.

We calculate the predicted $\hat{y}$ as $\hat{y}_i = \frac{e^{z_i}}{\sum_j e^{z_j}}$, where $z_i$ are the logits.<br>
The derivative of the loss with respect to the correct class $c$'s logit $z_c$ is:
$$\frac{\partial L}{\partial z_c} = -\frac{1}{\hat{y}_c} \cdot \frac{\partial \hat{y}_c}{\partial z_c} = -\frac{1}{\hat{y}_c} \cdot \hat{y}_c(1-\hat{y}_c) = \hat{y}_c - 1 = \hat{y}_c - y_c$$

For all other classes $i \neq c$, the derivative is:
$$\frac{\partial L}{\partial z_i} = -\frac{1}{\hat{y}_c} \cdot \frac{\partial \hat{y}_c}{\partial z_i} = -\frac{1}{\hat{y}_c} \cdot (-\hat{y}_c\hat{y}_i) = \hat{y}_i - 0 = \hat{y}_i - y_i$$

If we go and combine the two (i.e. lifting the $i \neq c$ constaint), we get:
$$\frac{\partial L}{\partial z_i} = \hat{y}_i - y_i$$

- For the correct class, this expression subtracts $1$ from the predicted probability of the correct class $\hat{y}_c$
- For incorrect classes, this expression simply returns the predicted probability of the class $\hat{y}_i$

But we're not fully done just yet. We produce a single cross entropy loss value for our entire batch.<br>
We do this by having `cross_entropy` return the mean of the losses for each sample in the batch.

Therefore, we have to implement a normalized version of the gradient above, which is:
$$\frac{\partial L}{\partial z_i} = \frac{1}{n} \sum_{j=1}^{n} (\hat{y}_{ij} - y_{ij})$$
where $n$ is the number of samples in the batch, $\hat{y}_{i}$ is the predicted probability of each class $i$ and $y_{i}$ is the true label of this class $i$, i.e. the one-hot encoding for example $j$.

One last thing, the probabilities $\hat{y}_{ij}$ were given by `probs` in our earlier implementation: `probs = counts * counts_sum_inv`.<br>
As this has been abstracted away, we will have to make `probs` ourselves by using the softmax function on the logits, explicitly.

In [11]:
## Backward pass
dlogits = (F.softmax(logits, dim=1) - F.one_hot(Yb, num_classes=logits.shape[1])) / n
cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9 (we got 5.58e-9)

logits          | exact: False | approximate: True  | maxdiff: 6.05359673500061e-09


## Exercise 3 - Backprop through batchnorm but all in one go

**Objective:** To complete this challenge, look at the mathematical expression of the output of batchnorm,<br>
take the derivative w.r.t. its input, simplify the expression, and just write it out.<br><br>
[BatchNorm paper](https://arxiv.org/abs/1502.03167) for reference.

In [12]:
# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


The task is to implement the gradient of the batchnorm layer with respect to its input.

Let's have another look at the BatchNorm definition:<br>
![](../004%20-%20Makemore%203%20-%20Activations,%20BatchNorm/img/batch_norm_recipe.PNG)

Fundamentally, this let's us express the output $y_i$ as:<br>
$$y_i = \gamma \cdot \frac{x_i - \mu_{\mathcal{B}}}{\sqrt{\sigma²_\mathcal{B}+\epsilon}} + \beta$$

The gradient with respect to $x_i$ can be split into three terms:

1. The direct effect of $x_i$ on $y_i$: $\frac{\gamma}{\sqrt{\sigma^2_\mathcal{B} + \epsilon}} \cdot \frac{\partial L}{\partial y_i}$
2. The effect of the mean $\mu_{\mathcal{B}}$ on $y_i$: $-\frac{\gamma}{\sqrt{\sigma^2_\mathcal{B} + \epsilon}} \cdot \frac{1}{n} \sum_{j=1}^n \frac{\partial L}{\partial y_j}$
3. The effect of the variance $\sigma^2_\mathcal{B}$ on $y_i$: $-\frac{\gamma}{\sqrt{\sigma^2_\mathcal{B} + \epsilon}} \cdot \frac{x_i - \mu_\mathcal{B}}{\sigma^2_\mathcal{B} + \epsilon} \cdot \frac{1}{n} \sum_{j=1}^n \frac{\partial L}{\partial y_j} (x_j - \mu_\mathcal{B})$

- `dbndiff.clone()` is a deep copy of the gradient of the batchnorm layer with respect to its input, i.e. step 1 above.
- `(1.0 / n) * (torch.ones_like(hprebn) * dbnmeani)` combines terms 2 and 3, as `dbnmeani` is the sum of terms 2 and 3 for all $i$, which is then distributed equally across all inputs due to the chain rule through the mean calculation.

Condensing all of this into a single line, we get:

In [13]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

dhprebn = dbndiff.clone() + (1.0 / n) * (torch.ones_like(hprebn) * dbnmeani)
cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, maxdiff is expected to be 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


## Exercise 4 - Putting it all together!

**Objective:** Train the MLP neural net with your own backward pass.

In [22]:
# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print('Parameter Count:', sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written
with torch.no_grad():

  # kick off optimization
  for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    # Use this for correctness comparisons, comment it out later!
    #for p in parameters:
      #p.grad = None
    #loss.backward()

    # manual backprop! #swole_doge_meme
    # -----------------
    dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None

    # I opt for this writing style to
    # better connect the dots with earlier explanations
    dlogits = (F.softmax(logits, dim=1) - F.one_hot(Yb, num_classes=logits.shape[1])) / n
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)
    dhpreact = (1.0 - h ** 2) * dh
    dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
    dbnbias = dhpreact.sum(0, keepdim=True)
    dbnraw = bngain * dhpreact
    dbnvar_inv = ((hprebn - bnmean) * dbnraw).sum(0, keepdim=True)
    dbndiff = bnvar_inv * dbnraw
    dbnvar = (-0.5 * (bnvar + 1e-5) ** (-1.5)) * dbnvar_inv
    dbndiff2 = (1.0 / (n-1)) * torch.ones_like((hprebn - bnmean)**2) * dbnvar
    dbndiff += 2 * (hprebn - bnmean) * dbndiff2
    dbnmeani = (-1.0) * dbndiff.sum(0, keepdim=True)
    dhprebn = dbndiff.clone() + (1.0 / n) * (torch.ones_like(hprebn) * dbnmeani)
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(0)
    demb = dembcat.view(n, -1, emb.shape[2])
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
        for j in range(Xb.shape[1]):
            ix = Xb[k, j]
            dC[ix] += demb[k, j]

    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      # p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
      p.data += -lr * grad # new way of swole doge: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

    # Undo early breaking when you're ready to train the full net
    #if i >= 100:
      #break

Parameter Count: 12297
      0/ 200000: 3.8101
  10000/ 200000: 2.1927
  20000/ 200000: 2.3847
  30000/ 200000: 2.4974
  40000/ 200000: 1.9947
  50000/ 200000: 2.2664
  60000/ 200000: 2.3816
  70000/ 200000: 2.0237
  80000/ 200000: 2.3433
  90000/ 200000: 2.1643
 100000/ 200000: 1.9384
 110000/ 200000: 2.3152
 120000/ 200000: 2.0338
 130000/ 200000: 2.4350
 140000/ 200000: 2.2667
 150000/ 200000: 2.0973
 160000/ 200000: 2.0412
 170000/ 200000: 1.8830
 180000/ 200000: 2.0602
 190000/ 200000: 1.9895


In [20]:
# Useful for checking your gradients
for p,g in zip(parameters, grads):
    cmp(str(tuple(p.shape)), g, p)

(27, 10)        | exact: False | approximate: True  | maxdiff: 1.1641532182693481e-08
(30, 200)       | exact: False | approximate: True  | maxdiff: 8.381903171539307e-09
(200,)          | exact: False | approximate: True  | maxdiff: 8.381903171539307e-09
(200, 27)       | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
(27,)           | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
(1, 200)        | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09
(1, 200)        | exact: False | approximate: True  | maxdiff: 4.190951585769653e-09


In [23]:
# Calibrate the batch norm at the end of training
with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [24]:
# Evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.0716681480407715
val 2.109276294708252


Andrej's baseline:<br>
- train $2.0718822479248047$,
- val $2.1162495613098145$.

In [25]:
# sample from the model
for _ in range(20):
    out = []
    context = [0] * block_size # initialize with all zeros
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

mon.
ammyah.
see.
mad.
rylle.
emmani.
jarlee.
adered.
elin.
shy.
jenleigh.
sana.
arleigh.
malaia.
nosalbergahirael.
kin.
renlynn.
novana.
ubrence.
ryyah.


<center>Notebook by <a href="https://github.com/mk2112" target="_blank">mk2112</a>.</center>